## Train GPT on addition

Train a GPT model on a dedicated addition dataset to see if a Transformer can learn to add.

In [1]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [2]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [11]:
import numpy as np
import torch
import string
import os
from tqdm import tqdm
import torch.nn as nn
from torch.nn import functional as F
from mingpt.md import MemData
from mingpt.marker_dataset import MarkerDataset
from mingpt.math_dataset import MathDataset
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# create a dataset 
easy = 'run/numbers__place_value.txt'
medium = 'run/numbers__is_prime.txt'
hard = 'run/numbers__list_prime_factors.txt'

In [55]:
!rm -rf run
!cp -r data run

In [56]:
memory_slots = 7
MD = MemData(memory_slots)
MD.initiate_mem_slot_data(hard)

In [57]:
# create a dataset 
easy_test = 'run/test_numbers__place_value.txt'
medium_test = 'run/test_numbers__is_prime.txt'
hard_test = 'run/test_numbers__list_prime_factors.txt'
easy_train = 'run/train_buffer_numbers__place_value.txt'
medium_train = 'run/train_buffer_numbers__is_prime.txt'
hard_train = 'run/train_buffer_numbers__list_prime_factors.txt'

In [58]:
train_dataset = MathDataset(fname=hard_train, MD=MD)
test_dataset = MathDataset(fname=hard_test, MD=MD)

In [17]:
MD.block_size

284

In [53]:
train_dataset[1000]

(tensor([61, 84, 94, 95,  7, 95, 83, 80,  7, 91, 93, 84, 88, 80,  7, 81, 76, 78,
         95, 90, 93, 94,  7, 90, 81,  7, 47, 49, 42, 48, 40, 42, 21,  1,  2,  2,
          2,  2,  2,  2,  2,  5, 42, 19,  7, 42, 49, 19,  7, 41, 43, 46, 46, 49,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,

In [19]:
from mingpt.model import GPT, GPTConfig, GPT1Config

# initialize a baby GPT model
mconf = GPTConfig(MD.vocab_size, MD.block_size, 
                  n_layer=2, n_head=4, n_embd=128)
model = GPT(mconf)

11/21/2020 09:09:31 - INFO - mingpt.model -   number of parameters: 4.592640e+05


In [ ]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=1, batch_size=512, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=1024, final_tokens=50*len(train_dataset)*(14+1),
                      num_workers=0)
trainer = Trainer(model, train_dataset, test_dataset, tconf)
trainer.train()

epoch 1 iter 292: train loss 1.23760. lr 5.999615e-04:  25%|██▌       | 293/1172 [01:11<03:34,  4.10it/s]

In [44]:
trainer.save_checkpoint()

11/21/2020 09:52:38 - INFO - mingpt.trainer -   saving model.pth


In [60]:
# now let's give the trained model an addition exam
from torch.utils.data.dataloader import DataLoader
from mingpt.examiner import Examiner
examiner = Examiner(MD)
examiner.exam(hard_train, train_dataset, trainer)

Iiter 9999: train loss 0.73722.:   2%|▏         | 9999/600000 [00:31<31:19, 313.88it/s]  


Final score: 73/9902 = 0.74% correct
Saving new files to disk...


In [ ]:
# training set: how well did we memorize?
examples = give_exam(test_dataset, batch_size=1, max_batches=-1)
print("Q: %s\nX:%s\nO:%s\n" % (examples[0][0], examples[0][1] , examples[0][2]))

In [ ]:
for item in examples:
    print("Question:", item[0])
    print("X:", item[1])
    print("Out:", item[2])



In [ ]:
# test set: how well did we generalize?
give_exam(train_dataset, batch_size=1024, max_batches=1)

In [ ]:
# well that's amusing... our model learned everything except 55 + 45

In [ ]:
import itertools as it

In [ ]:
f = ['-1', '-1', '2', '1', '1']

it.takewhile(lambda x: x!='2', f)

In [ ]:
f